# 获取交易所历史数据

交易所公布了历史数据（日线）以及持仓量等等不少数据，手工下载非常消耗时间。

我们可是写一个简单的爬虫来辅助处理。

需要使用 requests 库，这是一个第三方库，安装：

`pip install requests`

In [1]:
# 处理 JupyterLab 的路径问题。
import sys
sys.path.insert(0, '..')

## 上期所

在[上期所官网](http://www.shfe.com.cn/)点击顶部菜单（如下图中红框所示）：

![上期所历史数据获取1](../pictures/shfe_data_1.png)

然后在出现的页面中，更换下拉框中的年份并点击按钮（如下图中红框所示）。

![上期所历史数据获取2](../pictures/shfe_data_2.png)

多试几次，可以发现历史数据的url是：

`
  http://www.shfe.com.cn/historyData/MarketData_Year_XXXX.zip
`

其中 XXXX 是四位数的年份数字。

于是我们可以用以下代码来获得上期所历史数据：

In [2]:
from pathlib import Path
import datetime as dt

import requests

from src.utility import NOTEBOOK_PATH


def download_shfe_history_data(year: int, save_path: Path) -> None:
    """
    下载上海期货交易所（上期所，SHFE）的历史数据。
    :param year: 需要下载数据的年份。
    :param save_path: 保存的位置。
    :return: None.
    """

    # 上期所历史数据 url.
    url: str = 'http://www.shfe.com.cn/historyData/MarketData_Year_{year:4d}.zip'

    # 上期所历史数据从 2009 年开始提供。
    start_year: int = 2009
    
    # 如果参数 <year> 小于 2009 或者大于当前年份，引发异常。
    if year < start_year:
        raise ValueError(f'上期所历史数据自{start_year:4d}年起提供。')
    if year > dt.date.today().year:
        raise ValueError(f'{year}年是未来日期。')

    # 如果参数<save_path>不存在，引发异常。
    if not save_path.exists():
        raise FileNotFoundError(f'目录 {save_path} 不存在。')
    
    # 下载。
    response = requests.get(url.format(year=year))
    
    # 如果下载不顺利，引发异常。
    if response.status_code != 200:
        raise requests.exceptions.HTTPError(f'下载 <{url.format(year=year)}> 时发生错误。')
    
    # 保存文件。
    with open(save_path.joinpath(f'SHFE_{year:4d}.zip'), 'wb') as f:
        f.write(response.content)

# 保存位置为代码仓库下的 temp/ 文件夹。
download_shfe_history_data(2020, NOTEBOOK_PATH.parent.joinpath('temp'))

你应该可以在代码仓库的 temp/ 文件夹下面看到名为 <SHFE_2020.zip> 文件。

我们可以自动化下载从2009年（上期所开始提供数据的年份）到现在的所有历史数据：

In [3]:
def download_all_shfe_history_data(save_path: Path) -> None:
    """
    下载上海期货交易所（上期所，SHFE）的全部历史数据。
    :param save_path: 保存的位置。
    :return: None.
    """
    start_year: int = 2009
    this_year: int = dt.date.today().year
    for year in range(start_year, this_year + 1):
        download_shfe_history_data(year, save_path)

download_all_shfe_history_data(NOTEBOOK_PATH.parent.joinpath('temp'))

很迅速，而且有条理，比手工下载强多了。

## 中金所

[中金所官网](http://www.cffex.com.cn/)顶部菜单“数据” -> “历史数据下载”。

中金所数据是按月提供的。

月份有点讨厌，因为年份比较简单，就是一个数字。月份的话，需要提供年份、月份两个数字，而这两个数字的逻辑关系判断很繁琐。

简单起见，用 Python 的 datetime 模块下面的 date 数据类型，只是忽略日期。

In [7]:
def download_cffex_history_data(year: int, month: int, save_path: Path) -> None:
    """
    下载中国金融期货交易所（中金所，CFFEX）的历史数据。
    :param year: 需要下载数据的年份。
    :param month: 需要下载数据的月份。
    :param save_path: 保存的位置。
    :return: None.
    """
    
    # 中金所历史数据 url 模板。
    url: str = 'http://www.cffex.com.cn/sj/historysj/{year:4d}{month:02d}/zip/{year:4d}{month:02d}.zip'

    # 中金所历史数据从 2010 年 4 月（股指期货）开始提供。
    start_year: int = 2010
    start_month: int = 4
    today: dt.date = dt.date.today()

    # 如果参数 <year> 和 <month> 不在合理范围，引发异常。
    if month < 1 or month > 12:
        raise ValueError(f'参数 <month> 取值范围在 [1, 12]。')
    if year < start_year or (year == start_year and month < start_month):
        raise ValueError(f'中金所历史数据自{start_year:4d}年{start_month:02d}月起开始提供。')
    if year > today.year or (year == today.year and month > today.month):
        raise ValueError(f'{year:4d}年{month:02d}月是未来日期。')

    # 如果参数<save_path>不存在，引发异常。
    if not save_path.exists():
        raise FileNotFoundError(f'目录 {save_path} 不存在。')

    # 下载。
    response = requests.get(url.format(year=year, month=month))
    
    # 如果下载不顺利，引发异常。
    if response.status_code != 200:
        raise requests.exceptions.HTTPError(
            f'下载 <{url.format(year=year, month=month)}> 时发生错误。'
        )

    # 保存文件。
    with open(save_path.joinpath(f'CFFEX_{year:4d}-{month:02d}.zip'), 'wb') as f:
        f.write(response.content)

# 保存位置为代码仓库下的 temp/ 文件夹。
download_cffex_history_data(
    year=2020,
    month=8,
    save_path=NOTEBOOK_PATH.parent.joinpath('temp')
)

自动化下载从2010年04月（股指期货上市，中金所开始提供数据的时间）到现在的所有历史数据：

In [8]:
def download_all_cffex_history_data(save_path: Path) -> None:
    """
    下载中国金融期货交易所（中金所，CFFEX）的全部历史数据。
    :param save_path: 保存的位置。
    :return: None.
    """
    start_year: int = 2010
    start_month: int = 4
    today: dt.date = dt.date.today()
    
    for year in range(start_year, today.year + 1):
        for month in range(1, 12 + 1):
            if year == 2010 and month < start_month:
                continue
            if year == today.year and month > today.month:
                break
            download_cffex_history_data(year, month, save_path)

download_all_cffex_history_data(NOTEBOOK_PATH.parent.joinpath('temp'))